In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Restaurant Finder")

LangSmith 추적을 시작합니다.
[프로젝트명]
Restaurant Finder


In [3]:
import os

# 현재 작업 디렉토리 경로 확인
current_dir = os.getcwd()
print("현재 작업 디렉토리:", current_dir)

현재 작업 디렉토리: e:\STUDY\Python\Capstone\Travel Agent


In [4]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# CSV 로더 생성
loader = CSVLoader(file_path="./data/food/음식테마거리/RAG_TEST_with_summary.csv")

# 데이터 로드
docs = loader.load()

print(len(docs))
print(docs[0])

100
page_content='﻿RSTR_ID: 112467
markdown_content: 요약: 더밥하우스는 부산광역시 사하구에 위치한 경양식 일반음식점입니다. 매일 10:00부터 22:00까지 운영하며, 토요일은 휴무입니다. 배달 서비스는 제공하지만, 주차 및 와이파이 등 편의 시설은 없습니다.

# 더밥하우스

## 기본 정보
- 위치: 부산광역시 사하구 낙동대로516번길 17
- 업종: 경양식
- 영업허가: 일반음식점
- 지역: 부산광역시 사하구

## 상세 설명
부산광역시 사하구에서 맛집을 찾으신다면 "더밥하우스"를 추천합니다.

## 운영 정보
- 영업시간: 매일 10:00~22:00
- 휴무일: 토요일
- 네이버 평점: 4.3299999

## 편의 시설
- 주차: N
- 와이파이: N
- 장애인 편의시설: N
- 반려동물 출입: N
- 화장실: N

## 메뉴 정보
- 외국어 메뉴 제공: N

## 주문 옵션
- 배달 서비스: Y
- 배달앱 주문: N
- 택배 판매: N
- 모바일 결제: N
- 온라인 예약: N' metadata={'source': './data/food/음식테마거리/RAG_TEST_with_summary.csv', 'row': 0}


In [5]:
from langchain.schema import Document

def prepare_restaurant_documents(docs):
    restaurant_docs = []
    for doc in docs:
        content = doc.page_content
        
        # RSTR_ID 값 추출 - BOM 문자를 명시적으로 처리
        rstr_id = None
        for line in content.split('\n'):
            if line.startswith('\ufeffRSTR_ID:') or line.startswith('RSTR_ID:'):
                rstr_id = int(line.split(':')[1].strip())
                break
        
        # RSTR_ID 줄을 제외한 나머지 내용만 포함
        content_lines = [line for line in content.split('\n') 
                        if not (line.startswith('\ufeffRSTR_ID:') or line.startswith('RSTR_ID:'))]
        filtered_content = '\n'.join(content_lines)
        
        restaurant_docs.append(Document(
            page_content=filtered_content.strip(),
            metadata={'RSTR_ID': rstr_id}
        ))
    
    return restaurant_docs

# 식당 문서 생성
restaurant_docs = prepare_restaurant_documents(docs)

print(f"식당 문서 수: {len(restaurant_docs)}")
print(f"\n첫 번째 식당 문서의 내용:\n{restaurant_docs[0].page_content}")
print(f"\n첫 번째 식당 문서의 메타데이터:\n{restaurant_docs[0].metadata}")

식당 문서 수: 100

첫 번째 식당 문서의 내용:
markdown_content: 요약: 더밥하우스는 부산광역시 사하구에 위치한 경양식 일반음식점입니다. 매일 10:00부터 22:00까지 운영하며, 토요일은 휴무입니다. 배달 서비스는 제공하지만, 주차 및 와이파이 등 편의 시설은 없습니다.

# 더밥하우스

## 기본 정보
- 위치: 부산광역시 사하구 낙동대로516번길 17
- 업종: 경양식
- 영업허가: 일반음식점
- 지역: 부산광역시 사하구

## 상세 설명
부산광역시 사하구에서 맛집을 찾으신다면 "더밥하우스"를 추천합니다.

## 운영 정보
- 영업시간: 매일 10:00~22:00
- 휴무일: 토요일
- 네이버 평점: 4.3299999

## 편의 시설
- 주차: N
- 와이파이: N
- 장애인 편의시설: N
- 반려동물 출입: N
- 화장실: N

## 메뉴 정보
- 외국어 메뉴 제공: N

## 주문 옵션
- 배달 서비스: Y
- 배달앱 주문: N
- 택배 판매: N
- 모바일 결제: N
- 온라인 예약: N

첫 번째 식당 문서의 메타데이터:
{'RSTR_ID': 112467}


In [6]:
# 벡터스토어 생성
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(
    documents=restaurant_docs, 
    embedding=OpenAIEmbeddings()
)

# 검색기(retriever) 생성
retriever = vectorstore.as_retriever()

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

# 프롬프트 템플릿 정의 - 여러 식당을 추천하도록
restaurant_finder_template = """
당신은 레스토랑 추천 AI입니다. 주어진 맥락을 바탕으로 사용자의 질문에 답변해주세요.

다음은 레스토랑에 대한 정보입니다:
{restaurant_info}

사용자 질문: {user_request}

다음 지침을 따라 답변해주세요:
1. 조건에 맞는 식당을 2-3개 추천해주세요.
2. 각 식당의 이름을 정확히 큰따옴표로 감싸서 언급해주세요. (예: "더밥하우스")
3. 각 식당의 주요 특징을 간단히 설명해주세요.

레스토랑 정보를 바탕으로 사용자의 질문에 답변해주세요.
"""

prompt = PromptTemplate.from_template(restaurant_finder_template)

# LLM 설정
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.2)

In [8]:
# RAG 체인 생성
rag_chain = (
    {
        "restaurant_info": retriever,
        "user_request": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

# 체인 테스트
user_request = "부산에서 네이버평점이 좋은 맛집을 추천해주세요."

answer = rag_chain.invoke(user_request)
print(answer)

부산에서 네이버 평점이 좋은 맛집을 추천드리겠습니다:

1. "한일돈까스": 부산광역시 사상구에 위치한 경양식 일반음식점으로, 네이버 평점이 4.4입니다. 매일 08:00부터 20:00까지 운영하며, 편의 시설은 제공되지 않습니다.

2. "가장맛있는족발족과의동침(엄궁점)": 부산광역시 사상구에 위치한 일반음식점으로, 네이버 평점이 4.17입니다. 매일 15:00부터 23:59까지 운영하며, 주차 및 와이파이 등의 편의 시설은 제공되지 않습니다.

3. "명륜진사갈비(당감점)": 부산광역시 부산진구에 위치한 식육(숯불구이) 전문 식당으로, 네이버 평점이 4.15입니다. 평일 16:30~23:00, 공휴일 12:00~23:00에 운영하며, 주차가 가능하고 배달 서비스도 제공합니다.

이 세 곳 모두 맛있는 음식을 즐길 수 있는 좋은 선택이 될 것입니다.


In [9]:
from typing import Dict, Any
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

def process_restaurant_response(retriever_output, llm_response: str) -> Dict[str, Any]:
    """
    LLM 응답에서 언급된 식당의 ID만 추출합니다.
    """
    mentioned_restaurants = {}
    # 검색된 문서들에서 식당 이름과 ID를 매핑
    for doc in retriever_output:
        content = doc.page_content
        rstr_id = doc.metadata['RSTR_ID']
        
        # 식당 이름 추출 (마크다운 첫 번째 헤딩에서)
        for line in content.split('\n'):
            if line.startswith('# '):
                restaurant_name = line.replace('# ', '').strip()
                mentioned_restaurants[restaurant_name] = rstr_id
                break
    
    # LLM 응답에서 언급된 식당 ID만 수집
    response_restaurant_ids = []
    for restaurant_name in mentioned_restaurants.keys():
        if restaurant_name in llm_response:
            response_restaurant_ids.append(mentioned_restaurants[restaurant_name])
    
    return {
        "answer": llm_response,
        "restaurant_ids": response_restaurant_ids
    }

# 프롬프트 템플릿 수정 - 여러 식당을 추천하도록
restaurant_finder_template = """
당신은 레스토랑 추천 AI입니다. 주어진 맥락을 바탕으로 사용자의 질문에 답변해주세요.

다음은 레스토랑에 대한 정보입니다:
{restaurant_info}

사용자 질문: {user_request}

다음 지침을 따라 답변해주세요:
1. 조건에 맞는 식당을 2-3개 추천해주세요.
2. 각 식당의 이름을 정확히 큰따옴표로 감싸서 언급해주세요. (예: "더밥하우스")
3. 각 식당의 주요 특징을 간단히 설명해주세요.

레스토랑 정보를 바탕으로 사용자의 질문에 답변해주세요.
"""

prompt = PromptTemplate.from_template(restaurant_finder_template)

# 나머지 체인 구성 코드는 동일...


# 검색 체인 구성
retrieval_chain = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

# 응답 생성 체인
response_chain = (
    {
        "restaurant_info": lambda x: x["context"],
        "user_request": lambda x: x["question"]
    }
    | prompt 
    | llm 
    | StrOutputParser()
)

# 최종 체인 구성
def final_processor(input_dict):
    question = input_dict["question"]
    docs = retriever.get_relevant_documents(question)
    response = response_chain.invoke(input_dict)
    return process_restaurant_response(docs, response)

rag_chain = retrieval_chain | final_processor

# 테스트
user_request = "부산에서 네이버평점이 좋은 맛집을 추천해주세요."
result = rag_chain.invoke(user_request)

print("사용자에게 보여줄 답변:")
print(result["answer"])
print("\n백엔드에 전달할 식당 ID:")
print(result["restaurant_ids"])

C:\Users\gulmo\AppData\Local\Temp\ipykernel_16624\1168002768.py:73: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  docs = retriever.get_relevant_documents(question)


사용자에게 보여줄 답변:
부산에서 네이버 평점이 좋은 맛집을 추천해드리겠습니다.

1. "한일돈까스": 부산광역시 사상구에 위치한 경양식 일반음식점으로, 네이버 평점이 4.4입니다. 매일 08:00부터 20:00까지 운영하며, 편의 시설은 제공되지 않습니다.

2. "가장맛있는족발족과의동침(엄궁점)": 부산광역시 사상구에 위치한 일반음식점으로, 네이버 평점이 4.17입니다. 매일 15:00부터 23:59까지 운영하며, 주차와 와이파이 등의 편의 시설은 제공되지 않습니다.

3. "명륜진사갈비(당감점)": 부산광역시 부산진구에 위치한 식육(숯불구이) 전문 식당으로, 네이버 평점이 4.15입니다. 평일에는 16:30부터 23:00까지, 공휴일에는 12:00부터 23:00까지 운영하며, 주차가 가능하고 배달 서비스도 제공됩니다.

백엔드에 전달할 식당 ID:
[579963, 687805, 29665]
